In [1]:
print('importing pandas')
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
print('importing svm')
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold

importing pandas


C:\Users\cep4u\AppData\Local\Continuum\anaconda3\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\cep4u\AppData\Local\Continuum\anaconda3\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
C:\Users\cep4u\AppData\Local\Continuum\anaconda3\lib\site-packages\numpy\.libs\libopenblas.XWYDX2IKJW2NMTWSFYNGFUWKQU3LYTCZ.gfortran-win_amd64.dll
  stacklevel=1)


importing svm


In [2]:
print('defining vectorizer')
tfidf_vectorizer = TfidfVectorizer(tokenizer=lambda x: [word_pos[0] for word_pos in eval(x)],lowercase=False,stop_words=None)

def tfidf_vectorize(data, column):
    tfidf_vectors = tfidf_vectorizer.fit_transform(data[column])

    return tfidf_vectors

defining vectorizer


In [3]:
def train_model(X, y):
    kfold = KFold(n_splits=5, shuffle=True, random_state=42)

    models = []
    
    for fold, (train_indices, val_indices) in enumerate(kfold.split(X, y)):
        X_train, y_train = X[train_indices], y[train_indices]
        X_val, y_val = X[val_indices], y[val_indices]

        model = svm.LinearSVC()

        model.fit(X, y)

        models.append(model)

    return models

In [4]:
print('importing data')
df = pd.read_csv("News Category Dataset Cleaned 20230416.csv")

importing data


In [5]:
categories = ['WELLNESS & HEALTH','ENTERTAINMENT','TRAVEL', 'STYLE & BEAUTY', 'PARENTING', 'QUEER VOICES', 'FOOD & DRINK']
data = df[df['category'].isin(categories)]

sampled_data = data.groupby('category', group_keys=False).apply(lambda x: x.sample(n=3000))
print("data imported")

label_encoder = LabelEncoder()
y = label_encoder.fit_transform(sampled_data['category'])
print("vectorize sparce")

X_sparse = tfidf_vectorize(sampled_data, 'wordnet_pos_desc')

data imported
vectorize sparce


In [6]:
X_sparse

<21000x31309 sparse matrix of type '<class 'numpy.float64'>'
	with 248843 stored elements in Compressed Sparse Row format>

In [7]:
# Withold validation set.
X_train_sparse, X_val_sparse, y_train, y_val = train_test_split(X_sparse, y, test_size=0.2, random_state=12)

print("data to array")
# Convert the sparse matrices to dense arrays
X = X_train_sparse.toarray()

data to array


In [8]:
print("train model")
# Train the support vector machine on the training data
models = train_model(X, y_train)

train model


In [9]:
acc_avg = []
X_val = X_val_sparse.toarray()
for model in models: # Evaluate the model on the testing data
    accuracy = model.score(X_val, y_val)
    acc_avg.append(accuracy)
    
print('Test accuracy:', sum(acc_avg)/len(acc_avg))

Test accuracy: 0.6323809523809524
